# Strands Agents SDK Demo

Welcome to this simple demo of the **Strands Agents SDK** - a simple-to-use, code-first framework for building agents.

This notebook will walk you through the basics of creating and using agents with Strands.

## What is Strands Agents?

Strands Agents is a framework that makes it easy to build AI agents in Python. It provides a simple API for creating agents that can understand and respond to questions, perform tasks, and interact with various tools and services.

## Prerequisites

Before we start, make sure you have the Strands Agents SDK installed.

In [ ]:
# Install Strands Agents SDK
# Uncomment the line below if you haven't installed it yet
# !pip install strands-agents

## 1. Creating Your First Agent

Let's start by creating a simple agent with default settings.

In [ ]:
from strands import Agent

# Create an agent with default settings
agent = Agent()

print("Agent created successfully!")
print(f"Agent type: {type(agent).__name__}")

## 2. Asking Your Agent a Question

Now let's ask our agent a simple question to see how it responds.

In [ ]:
# Ask the agent a question
response = agent("Tell me about agentic AI")
print("Agent's response:")
print(response)

## 3. Creating an Agent with Custom Configuration

Let's create a more customized agent with specific settings and behavior.

In [ ]:
# Create an agent with custom configuration
custom_agent = Agent(
    name="CodeHelper",
    description="A helpful coding assistant that specializes in Python programming",
    temperature=0.7,  # Controls randomness in responses
    max_tokens=500    # Limits response length
)

print(f"Custom agent '{custom_agent.name}' created!")
print(f"Description: {custom_agent.description}")

## 4. Using the Custom Agent

Let's test our custom coding assistant agent.

In [ ]:
# Ask the custom agent a coding question
coding_question = "How do I create a simple list comprehension in Python?"
coding_response = custom_agent(coding_question)

print(f"Question: {coding_question}")
print(f"\nCodeHelper's response:")
print(coding_response)

## 5. Conversation History

Strands agents can maintain conversation history, allowing for contextual conversations.

In [ ]:
# Start a conversation with context
print("Starting a contextual conversation...")

# First question
response1 = custom_agent("What is machine learning?")
print("Question 1: What is machine learning?")
print(f"Response 1: {response1}")

print("\n" + "="*50 + "\n")

# Follow-up question (should reference previous context)
response2 = custom_agent("Can you give me a simple Python example of it?")
print("Question 2: Can you give me a simple Python example of it?")
print(f"Response 2: {response2}")

## 6. Agent with Tools (Advanced)

Strands agents can be equipped with tools to perform specific tasks. Let's create an agent with some basic tools.

In [ ]:
import datetime
import random

# Define some simple tools
def get_current_time():
    """Get the current date and time."""
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def generate_random_number(min_val=1, max_val=100):
    """Generate a random number between min_val and max_val."""
    return random.randint(min_val, max_val)

def calculate_factorial(n):
    """Calculate the factorial of a number."""
    if n < 0:
        return "Factorial is not defined for negative numbers"
    elif n == 0 or n == 1:
        return 1
    else:
        result = 1
        for i in range(2, n + 1):
            result *= i
        return result

# Create an agent with tools
tool_agent = Agent(
    name="ToolAgent",
    description="An agent equipped with useful tools",
    tools=[
        get_current_time,
        generate_random_number,
        calculate_factorial
    ]
)

print("Tool-equipped agent created!")
print(f"Available tools: {[tool.__name__ for tool in tool_agent.tools]}")

## 7. Using Agent Tools

Now let's ask our tool-equipped agent to perform tasks using its tools.

In [ ]:
# Ask the agent to use its tools
questions_and_responses = [
    "What time is it right now?",
    "Can you generate a random number for me?",
    "What's the factorial of 5?",
    "Generate a random number between 50 and 100"
]

for question in questions_and_responses:
    response = tool_agent(question)
    print(f"Q: {question}")
    print(f"A: {response}")
    print("-" * 40)

## 8. Error Handling and Safety

Strands agents include built-in safety features and error handling.

In [ ]:
# Test error handling
try:
    # Try asking for something potentially problematic
    response = agent("Calculate the factorial of -5")
    print("Response:", response)
except Exception as e:
    print(f"Error handled gracefully: {e}")

# Test with empty input
try:
    response = agent("")
    print("Empty input response:", response)
except Exception as e:
    print(f"Empty input handled: {e}")

## 9. Agent Performance Monitoring

Let's look at some basic monitoring and evaluation of our agent's performance.

In [ ]:
import time

# Monitor response time
start_time = time.time()
response = agent("Explain what artificial intelligence is in simple terms")
end_time = time.time()

response_time = end_time - start_time
word_count = len(response.split())

print(f"Response time: {response_time:.2f} seconds")
print(f"Response word count: {word_count} words")
print(f"Words per second: {word_count/response_time:.2f}")
print("\nResponse:")
print(response)

## 10. Deployment Considerations

When deploying Strands agents, consider the following best practices:

In [ ]:
# Example of agent configuration for production
production_agent = Agent(
    name="ProductionAgent",
    description="A production-ready agent with optimized settings",
    temperature=0.3,  # Lower temperature for more consistent responses
    max_tokens=300,   # Reasonable token limit
    timeout=30,       # 30-second timeout
    rate_limit=100,   # Maximum 100 requests per minute
    logging=True,     # Enable logging for monitoring
    safety_checks=True # Enable additional safety checks
)

print("Production agent configuration:")
print(f"Name: {production_agent.name}")
print(f"Temperature: {production_agent.temperature}")
print(f"Max tokens: {production_agent.max_tokens}")
print(f"Timeout: {production_agent.timeout}s")
print(f"Rate limit: {production_agent.rate_limit} req/min")

## Conclusion

This notebook has demonstrated the key features of the Strands Agents SDK:

1. **Simple Agent Creation**: Easy to create agents with minimal code
2. **Customization**: Configure agents with specific parameters and behavior
3. **Tool Integration**: Equip agents with custom tools and functions
4. **Conversation Context**: Maintain conversation history for contextual interactions
5. **Error Handling**: Built-in safety and error handling mechanisms
6. **Performance Monitoring**: Track and evaluate agent performance
7. **Production Readiness**: Configuration options for deployment

### Next Steps

To learn more about Strands Agents:
- Visit the [official documentation](https://strandsagents.com/0.1.x/)
- Explore the User Guide sections:
  - Welcome
  - Quickstart
  - Concepts
  - Safety & Security
  - Observability & Evaluation
  - Deploy

Happy building with Strands Agents! 🚀